# User Service Integration Tests

This notebook tests the user service endpoints through the API Gateway.

**Endpoints tested:**
- GET `/api/v1/users/me` - Get current user profile
- PUT `/api/v1/users/me` - Update user profile (full replace)
- PATCH `/api/v1/users/me` - Partial update user profile
- POST `/api/v1/users/pets` - Create new pet profile
- GET `/api/v1/users/pets` - List user's pets
- GET `/api/v1/users/pets/{id}` - Get specific pet
- PUT/PATCH `/api/v1/users/pets/{id}` - Update pet
- DELETE `/api/v1/users/pets/{id}` - Delete pet

**Prerequisites:** Must be authenticated (run auth service tests first to login).

## Setup and Configuration

In [1]:
import requests
import json
from datetime import datetime, date
from rich import print as rprint
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.json import JSON
import time

console = Console()

# API Gateway URL
BASE_URL = "http://localhost:8001"
USER_BASE = f"{BASE_URL}/api/v1/users"
PET_BASE = f"{BASE_URL}/api/v1/pets"

TEST_USER_EMAIL = "test_user@example.com"
TEST_USER_PASSWORD = "securepassword123"

# Create session (will need JWT cookies from auth)
session = requests.Session()

# Test results tracker
test_results = {}

console.print("[green]✓[/green] Setup complete", style="bold")
console.print(f"[blue]API Gateway:[/blue] {BASE_URL}")
console.print(f"[blue]User Endpoints:[/blue] {USER_BASE}")
console.print(f"[blue]Pet Endpoints:[/blue] {PET_BASE}")

✓ Setup complete

API Gateway: http://localhost:8001

User Endpoints: http://localhost:8001/api/v1/users

Pet Endpoints: http://localhost:8001/api/v1/pets

## Helper Functions

In [2]:
def print_response(response, title="Response"):
    """Pretty print HTTP response."""
    status_color = "green" if response.status_code < 300 else "yellow" if response.status_code < 400 else "red"
    
    table = Table(title=title, show_header=True, header_style="bold magenta")
    table.add_column("Property", style="cyan", width=20)
    table.add_column("Value", style="white")
    
    table.add_row("Status Code", f"[{status_color}]{response.status_code}[/{status_color}]")
    table.add_row("URL", response.url)
    table.add_row("Time", f"{response.elapsed.total_seconds():.3f}s")
    
    console.print(table)
    
    try:
        json_data = response.json()
        console.print("\n[bold]Response Body:[/bold]")
        console.print(JSON(json.dumps(json_data, indent=2)))
    except:
        console.print(f"\n[yellow]Raw Response:[/yellow] {response.text}")
    
    console.print("\n" + "="*80 + "\n")
    return response


def test_endpoint(method, url, data=None, expect_success=True, test_name=None, **kwargs):
    """Test an endpoint and print results."""
    try:
        response = session.request(method, url, json=data, **kwargs)
        print_response(response, f"{method.upper()} {url}")
        
        passed = False
        if expect_success and response.status_code >= 400:
            console.print("[red]✗ Expected success but got error[/red]", style="bold")
        elif not expect_success and response.status_code < 400:
            console.print("[yellow]⚠ Expected error but got success[/yellow]", style="bold")
        else:
            console.print("[green]✓ Response matches expectation[/green]", style="bold")
            passed = True
        
        # Record test result if test_name provided
        if test_name:
            test_results[test_name] = {
                'passed': passed,
                'status_code': response.status_code,
                'expected_success': expect_success
            }
        
        return response
    except requests.exceptions.RequestException as e:
        console.print(f"[red]✗ Request failed: {e}[/red]", style="bold")
        if test_name:
            test_results[test_name] = {'passed': False, 'status_code': 'ERROR', 'expected_success': expect_success}
        return None


console.print("[green]✓[/green] Helper functions loaded", style="bold")

✓ Helper functions loaded

## 1. Authentication

Login to get JWT cookies for authenticated requests.

In [3]:
console.print("\n[bold cyan]Registering Test User...[/bold cyan]\n")

register_data = {
    "email": TEST_USER_EMAIL,
    "password": TEST_USER_PASSWORD
}

response = test_endpoint("POST", f"{BASE_URL}/api/v1/auth/register", data=register_data, expect_success=True, test_name="User Registration")

console.print("\n[bold cyan]Authenticating...[/bold cyan]\n")

# Use existing test user or create new one
login_data = {
    "email": TEST_USER_EMAIL,
    "password": TEST_USER_PASSWORD
}

response = test_endpoint("POST", f"{BASE_URL}/api/v1/auth/login", data=login_data, test_name="Authentication")

if response and response.status_code == 200:
    console.print("[green]✓ Authentication successful[/green]\n", style="bold")
    authenticated = True
else:
    console.print("[red]✗ Authentication failed - run auth service tests first[/red]\n", style="bold")
    authenticated = False

Registering Test User...

           POST http://localhost:8001/api/v1/auth/register           
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 201                                        │
│ URL                  │ http://localhost:8001/api/v1/auth/register │
│ Time                 │ 0.400s                                     │
└──────────────────────┴────────────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "user": {
      "id": "b2e0bee8-eaf3-455f-94d2-1efe190c6035",
      "email": "test_user@example.com",
      "first_name": "",
      "last_name": "",
      "role": "user",
      "is_verified": true
    }
  },
  "error": null,
  "timestamp": "2026-01-27T18:53:54.262832Z"
}

================================================================================

...

Response Body:

{
  "success": true,
  "data": {
    "user": {
      "id": "b2e0bee8-eaf3-455f-94d2-1efe190c6035",
      "email": "test_user@example.com",
      "first_name": "",
      "last_name": "",
      "role": "user",
      "is_verified": true
    }
  },
  "error": null,
  "timestamp": "2026-01-27T18:53:54.665693Z"
}

================================================================================

✓ Response matches expectation

✓ Authentication successful

## 2. Get User Profile

Retrieve the current user's profile information.

In [4]:
console.print("\n[bold cyan]Testing Get User Profile...[/bold cyan]\n")

response = test_endpoint("GET", f"{USER_BASE}/me", test_name="Get User Profile")

if response and response.status_code == 200:
    console.print("[green]✓ Profile retrieved successfully[/green]\n", style="bold")
    current_profile = response.json().get("data", {})
else:
    console.print("[red]✗ Failed to retrieve profile[/red]\n", style="bold")
    current_profile = None

Testing Get User Profile...

           GET http://localhost:8001/api/v1/users/me            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                                   │
│ URL                  │ http://localhost:8001/api/v1/users/me │
│ Time                 │ 0.036s                                │
└──────────────────────┴───────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "id": "570500ac-8e34-4518-8787-511b4a1eff53",
    "user_id": "b2e0bee8-eaf3-455f-94d2-1efe190c6035",
    "phone": "",
    "address": null,
    "preferences": {},
    "created_at": "2026-01-27T18:53:55.116480Z",
    "updated_at": "2026-01-27T18:53:55.116507Z"
  },
  "error": null,
  "timestamp": "2026-01-27T18:53:55.121533Z"
}

================================================================================

✓ Response matches expectation

✓ Profile retrieved successfully

## 3. Update User Profile (PATCH)

Partially update user profile fields.

In [5]:
console.print("\n[bold cyan]Testing Partial Profile Update (PATCH)...[/bold cyan]\n")

update_data = {
    "phone": "+1-555-0123",
    "address": "123 Test Street, Test City, TC 12345"
}

response = test_endpoint("PATCH", f"{USER_BASE}/me", data=update_data, test_name="Update Profile (PATCH)")

if response and response.status_code == 200:
    console.print("[green]✓ Profile updated successfully[/green]\n", style="bold")
    updated_profile = response.json().get("data", {})
else:
    console.print("[red]✗ Profile update failed[/red]\n", style="bold")

Testing Partial Profile Update (PATCH)...

          PATCH http://localhost:8001/api/v1/users/me           
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                                   │
│ URL                  │ http://localhost:8001/api/v1/users/me │
│ Time                 │ 0.037s                                │
└──────────────────────┴───────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "id": "570500ac-8e34-4518-8787-511b4a1eff53",
    "user_id": "b2e0bee8-eaf3-455f-94d2-1efe190c6035",
    "phone": "+1-555-0123",
    "address": "123 Test Street, Test City, TC 12345",
    "preferences": {},
    "created_at": "2026-01-27T18:53:55.116480Z",
    "updated_at": "2026-01-27T18:53:55.182484Z"
  },
  "error": null,
  "timestamp": "2026-01-27T18:53:55.187694Z"
}

================================================================================

✓ Response matches expectation

✓ Profile updated successfully

## 4. Create Pet Profile

Add a new pet to the user's account.

In [6]:
console.print("\n[bold cyan]Testing Create Pet Profile...[/bold cyan]\n")

pet_data = {
    "name": "Max",
    "species": "dog",
    "breed": "Golden Retriever",
    "age": 3,
    "weight": 30.5,
    "health_conditions": "Hip dysplasia - under monitoring"
}

response = test_endpoint("POST", f"{PET_BASE}", data=pet_data, test_name="Create Pet")

if response and response.status_code in [200, 201]:
    console.print("[green]✓ Pet created successfully[/green]\n", style="bold")
    created_pet = response.json().get("data", {})
    pet_id = created_pet.get("id")
else:
    console.print("[red]✗ Pet creation failed[/red]\n", style="bold")
    pet_id = None

Testing Create Pet Profile...

           POST http://localhost:8001/api/v1/pets           
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 201                               │
│ URL                  │ http://localhost:8001/api/v1/pets │
│ Time                 │ 0.023s                            │
└──────────────────────┴───────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "id": "a57f4ab4-eeee-4781-8e36-221e8c769bd3",
    "user_id": "b2e0bee8-eaf3-455f-94d2-1efe190c6035",
    "name": "Max",
    "breed": "Golden Retriever",
    "breed_confidence": null,
    "species": "dog",
    "age": null,
    "weight": null,
    "health_conditions": [],
    "image_url": null,
    "created_at": "2026-01-27T18:53:55.236970Z",
    "updated_at": "2026-01-27T18:53:55.236987Z"
  },
  "error": null,
  "timestamp": "2026-01-27T18:53:55.241318Z"
}

================================================================================

✓ Response matches expectation

✓ Pet created successfully

## 5. List All Pets

Retrieve all pets for the current user.

In [7]:
console.print("\n[bold cyan]Testing List All Pets...[/bold cyan]\n")

response = test_endpoint("GET", f"{PET_BASE}", test_name="List Pets")

if response and response.status_code == 200:
    pets_data = response.json().get("data", [])
    console.print(f"[green]✓ Found {len(pets_data)} pet(s)[/green]\n", style="bold")
    
    # Display pets in table
    if pets_data:
        pets_table = Table(title="User's Pets", show_header=True, header_style="bold magenta")
        pets_table.add_column("ID", style="cyan")
        pets_table.add_column("Name", style="green")
        pets_table.add_column("Species", style="yellow")
        pets_table.add_column("Breed", style="blue")
        pets_table.add_column("Age", style="white")
        
        for pet in pets_data:
            pets_table.add_row(
                str(pet.get("id", "")),
                pet.get("name", ""),
                pet.get("species", ""),
                pet.get("breed", ""),
                str(pet.get("age", ""))
            )
        
        console.print(pets_table)
        console.print()
else:
    console.print("[red]✗ Failed to list pets[/red]\n", style="bold")

Testing List All Pets...

           GET http://localhost:8001/api/v1/pets            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                               │
│ URL                  │ http://localhost:8001/api/v1/pets │
│ Time                 │ 0.062s                            │
└──────────────────────┴───────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": [
    {
      "id": "a57f4ab4-eeee-4781-8e36-221e8c769bd3",
      "user_id": "b2e0bee8-eaf3-455f-94d2-1efe190c6035",
      "name": "Max",
      "breed": "Golden Retriever",
      "breed_confidence": null,
      "species": "dog",
      "age": null,
      "weight": null,
      "health_conditions": [],
      "image_url": null,
      "created_at": "2026-01-27T18:53:55.236970Z",
      "updated_at": "2026-01-27T18:53:55.236987Z"
    }
  ],
  "error": null,
  "timestamp": "2026-01-27T18:53:55.264971Z"
}

================================================================================

✓ Response matches expectation

✓ Found 1 pet(s)

                                    User's Pets                                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━┓
┃ ID                                   ┃ Name ┃ Species ┃ Breed            ┃ Age  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━┩
│ a57f4ab4-eeee-4781-8e36-221e8c769bd3 │ Max  │ dog     │ Golden Retriever │ None │
└──────────────────────────────────────┴──────┴─────────┴──────────────────┴──────┘

## 6. Get Specific Pet

Retrieve details for a specific pet.

In [8]:
if pet_id:
    console.print(f"\n[bold cyan]Testing Get Pet by ID ({pet_id})...[/bold cyan]\n")
    
    response = test_endpoint("GET", f"{PET_BASE}/{pet_id}", test_name="Get Specific Pet")
    
    if response and response.status_code == 200:
        console.print("[green]✓ Pet details retrieved[/green]\n", style="bold")
    else:
        console.print("[red]✗ Failed to retrieve pet[/red]\n", style="bold")
else:
    console.print("[yellow]⚠ Skipping - no pet ID available[/yellow]\n")

Testing Get Pet by ID (a57f4ab4-eeee-4781-8e36-221e8c769bd3)...

           GET http://localhost:8001/api/v1/pets/a57f4ab4-eeee-4781-8e36-221e8c769bd3            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                                                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                                                                    │
│ URL                  │ http://localhost:8001/api/v1/pets/a57f4ab4-eeee-4781-8e36-221e8c769bd3 │
│ Time                 │ 0.033s                                                                 │
└──────────────────────┴────────────────────────────────────────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "id": "a57f4ab4-eeee-4781-8e36-221e8c769bd3",
    "user_id": "b2e0bee8-eaf3-455f-94d2-1efe190c6035",
    "name": "Max",
    "breed": "Golden Retriever",
    "breed_confidence": null,
    "species": "dog",
    "age": null,
    "weight": null,
    "health_conditions": [],
    "image_url": null,
    "created_at": "2026-01-27T18:53:55.236970Z",
    "updated_at": "2026-01-27T18:53:55.236987Z"
  },
  "error": null,
  "timestamp": "2026-01-27T18:53:55.382736Z"
}

================================================================================

✓ Response matches expectation

✓ Pet details retrieved

## 7. Update Pet (PATCH)

Partially update pet information.

In [9]:
if pet_id:
    console.print(f"\n[bold cyan]Testing Update Pet ({pet_id})...[/bold cyan]\n")
    
    pet_update = {
        "age": 4,
        "weight": 32.0,
        "health_conditions": "Hip dysplasia - improved with supplements"
    }
    
    response = test_endpoint("PATCH", f"{PET_BASE}/{pet_id}", data=pet_update, test_name="Update Pet (PATCH)")
    
    if response and response.status_code == 200:
        console.print("[green]✓ Pet updated successfully[/green]\n", style="bold")
    else:
        console.print("[red]✗ Pet update failed[/red]\n", style="bold")
else:
    console.print("[yellow]⚠ Skipping - no pet ID available[/yellow]\n")

Testing Update Pet (a57f4ab4-eeee-4781-8e36-221e8c769bd3)...

          PATCH http://localhost:8001/api/v1/pets/a57f4ab4-eeee-4781-8e36-221e8c769bd3           
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                                                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                                                                    │
│ URL                  │ http://localhost:8001/api/v1/pets/a57f4ab4-eeee-4781-8e36-221e8c769bd3 │
│ Time                 │ 0.037s                                                                 │
└──────────────────────┴────────────────────────────────────────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "id": "a57f4ab4-eeee-4781-8e36-221e8c769bd3",
    "user_id": "b2e0bee8-eaf3-455f-94d2-1efe190c6035",
    "name": "Max",
    "breed": "Golden Retriever",
    "breed_confidence": null,
    "species": "dog",
    "age": 4,
    "weight": 32.0,
    "health_conditions": "Hip dysplasia - improved with supplements",
    "image_url": null,
    "created_at": "2026-01-27T18:53:55.236970Z",
    "updated_at": "2026-01-27T18:53:55.444218Z"
  },
  "error": null,
  "timestamp": "2026-01-27T18:53:55.448148Z"
}

================================================================================

✓ Response matches expectation

✓ Pet updated successfully

## 8. Create Multiple Pets

Test creating multiple pets with different species.

In [10]:
console.print("\n[bold cyan]Testing Multiple Pet Creation...[/bold cyan]\n")

test_pets = [
    {
        "name": "Whiskers",
        "species": "cat",
        "breed": "Persian",
        "age": 2,
        "weight": 4.5
    },
    {
        "name": "Buddy",
        "species": "dog",
        "breed": "Labrador",
        "age": 5,
        "weight": 35.0
    },
    {
        "name": "Tweety",
        "species": "other",
        "breed": "Parakeet (bird)",
        "age": 1,
        "weight": 0.03
    }
]

created_pet_ids = []
multiple_pets_success = True

for pet in test_pets:
    console.print(f"[blue]Creating pet: {pet['name']} ({pet['species']})[/blue]")
    response = session.post(f"{PET_BASE}", json=pet)
    
    if response.status_code in [200, 201]:
        pet_data = response.json().get("data", {})
        created_pet_ids.append(pet_data.get("id"))
        console.print(f"  [green]✓ Created with ID: {pet_data.get('id')}[/green]")
    else:
        console.print(f"  [red]✗ Failed to create {pet['name']}[/red]")
        multiple_pets_success = False

# Record test result
test_results['Multiple Pets'] = {
    'passed': multiple_pets_success,
    'status_code': 'OK' if multiple_pets_success else 'FAILED',
    'expected_success': True
}

console.print(f"\n[green]✓ Created {len(created_pet_ids)} additional pets[/green]\n", style="bold")

Testing Multiple Pet Creation...

Creating pet: Whiskers (cat)

✓ Created with ID: bf9affeb-44e4-429f-8450-3daa7bb5f2c8

Creating pet: Buddy (dog)

✓ Created with ID: 4573bb1c-6a49-4374-b67e-08e0da082a68

Creating pet: Tweety (other)

✓ Created with ID: 6f2c59b2-0c59-4106-9df3-adfa13e04066

✓ Created 3 additional pets

## 9. Test Authorization

Verify that users cannot access other users' pets.

In [11]:
console.print("\n[bold cyan]Testing Authorization (Access Other User's Pet)...[/bold cyan]\n")

# Try to access a pet with a non-existent ID (simulates other user's pet)
fake_pet_id = 99999

response = test_endpoint("GET", f"{PET_BASE}/{fake_pet_id}", expect_success=False, test_name="Authorization")

auth_test_passed = response and response.status_code in [404, 403]

if auth_test_passed:
    console.print("[green]✓ Correctly denied access to non-owned pet[/green]\n", style="bold")
else:
    console.print("[yellow]⚠ Authorization check did not work as expected[/yellow]\n", style="bold")

Testing Authorization (Access Other User's Pet)...

           GET http://localhost:8001/api/v1/pets/99999            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 404                                     │
│ URL                  │ http://localhost:8001/api/v1/pets/99999 │
│ Time                 │ 0.047s                                  │
└──────────────────────┴─────────────────────────────────────────┘

Response Body:

{
  "success": false,
  "data": null,
  "error": {
    "code": "NOT_FOUND",
    "message": "The requested resource was not found",
    "details": {}
  },
  "timestamp": "2026-01-27T18:53:55.734449"
}

================================================================================

✓ Response matches expectation

⚠ Authorization check did not work as expected

## 10. Test Input Validation

Verify that invalid data is rejected.

In [12]:
console.print("\n[bold cyan]Testing Input Validation...[/bold cyan]\n")

# Invalid pet data (missing required fields)
invalid_pet = {
    "name": "Invalid",
    # Missing species and breed
}

console.print("[blue]Sending invalid pet data (missing required fields)...[/blue]\n")
response = test_endpoint("POST", f"{PET_BASE}", data=invalid_pet, expect_success=False, test_name="Input Validation")

validation_test_passed = response.status_code == 422

if validation_test_passed:
    console.print("[green]✓ Correctly rejected invalid data[/green]\n", style="bold")
else:
    console.print("[yellow]⚠ Validation did not work as expected[/yellow]\n", style="bold")

Testing Input Validation...

Sending invalid pet data (missing required fields)...

           POST http://localhost:8001/api/v1/pets           
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 422                               │
│ URL                  │ http://localhost:8001/api/v1/pets │
│ Time                 │ 0.049s                            │
└──────────────────────┴───────────────────────────────────┘

Response Body:

{
  "success": false,
  "data": null,
  "error": {
    "code": "VALIDATION_ERROR",
    "message": "Invalid input",
    "details": {
      "species": [
        "This field is required"
      ]
    }
  },
  "timestamp": "2026-01-27T18:53:55.751126Z"
}

================================================================================

✓ Response matches expectation

✓ Correctly rejected invalid data

## 11. Cleanup - Delete Test Pets and User

Delete all pets created during testing.

In [13]:
console.print("\n[bold cyan]Cleaning Up Test Pets...[/bold cyan]\n")

# Collect all pet IDs
all_pet_ids = [pet_id] + created_pet_ids if pet_id else created_pet_ids

deleted_count = 0
cleanup_success = True

for pid in all_pet_ids:
    if pid:
        response = session.delete(f"{PET_BASE}/{pid}")
        if response.status_code in [200, 204]:
            console.print(f"[green]✓ Deleted pet ID: {pid}[/green]")
            deleted_count += 1
        else:
            console.print(f"[red]✗ Failed to delete pet ID: {pid}[/red]")
            cleanup_success = False

# Record test result
test_results['Cleanup'] = {
    'passed': cleanup_success,
    'status_code': 'OK' if cleanup_success else 'FAILED',
    'expected_success': True
}

console.print(f"\n[blue]Deleted {deleted_count} test pets[/blue]\n", style="bold")

# Delete test user
console.print("\n[bold cyan]Deleting test user...[/bold cyan]\n")
AUTH_BASE = f"{BASE_URL}/api/v1/auth"
delete_response = session.delete(f"{AUTH_BASE}/delete")

if delete_response.status_code == 200:
    data = delete_response.json()
    console.print(f"[green]✓ {data.get('data', {}).get('message', 'User deleted')}[/green]\n", style="bold")
else:
    console.print(f"[yellow]⚠ Failed to delete test user: {delete_response.status_code}[/yellow]\n", style="bold")


Cleaning Up Test Pets...

✓ Deleted pet ID: a57f4ab4-eeee-4781-8e36-221e8c769bd3

✓ Deleted pet ID: bf9affeb-44e4-429f-8450-3daa7bb5f2c8

✓ Deleted pet ID: 4573bb1c-6a49-4374-b67e-08e0da082a68

✓ Deleted pet ID: 6f2c59b2-0c59-4106-9df3-adfa13e04066

Deleted 4 test pets

Deleting test user...

✓ User account test_user@example.com deleted successfully

## 12. Test Summary

In [14]:
console.print("\n" + "="*80, style="bold")
console.print("[bold cyan]User Service Test Summary[/bold cyan]")
console.print("="*80 + "\n", style="bold")

summary_table = Table(show_header=True, header_style="bold magenta")
summary_table.add_column("Test", style="cyan", width=40)
summary_table.add_column("Status", style="white", width=20)
summary_table.add_column("Notes", style="white")

# Define expected test keys and their display info
test_info = {
    'Authentication': 'JWT cookies valid',
    'Get User Profile': 'Profile retrieved',
    'Update Profile (PATCH)': 'Partial update successful',
    'Create Pet': 'Pet profile created',
    'List Pets': 'All pets retrieved',
    'Get Specific Pet': 'Pet details retrieved',
    'Update Pet (PATCH)': 'Pet info updated',
    'Multiple Pets': f'Created {len(created_pet_ids)} pets',
    'Authorization': 'Access control working',
    'Input Validation': 'Invalid data rejected',
    'Cleanup': f'Deleted {deleted_count} pets'
}

# Build summary from actual test results
passed_count = 0
failed_count = 0

for test_name, notes in test_info.items():
    if test_name in test_results:
        result = test_results[test_name]
        if result['passed']:
            summary_table.add_row(test_name, "[green]✓ Pass[/green]", notes)
            passed_count += 1
        else:
            status_msg = f"✗ Fail (status: {result.get('status_code', 'N/A')})"
            summary_table.add_row(test_name, f"[red]{status_msg}[/red]", notes)
            failed_count += 1
    else:
        summary_table.add_row(test_name, "[yellow]⚠ Not Run[/yellow]", notes)
        failed_count += 1

console.print(summary_table)

# Overall result
if failed_count == 0:
    console.print(f"\n[bold green]All {passed_count} user service tests passed![/bold green]\n")
else:
    console.print(f"\n[bold red]{failed_count} test(s) failed, {passed_count} passed[/bold red]\n")

================================================================================

User Service Test Summary

================================================================================

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Test                                     ┃ Status               ┃ Notes                     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Authentication                           │ ✓ Pass               │ JWT cookies valid         │
│ Get User Profile                         │ ✓ Pass               │ Profile retrieved         │
│ Update Profile (PATCH)                   │ ✓ Pass               │ Partial update successful │
│ Create Pet                               │ ✓ Pass               │ Pet profile created       │
│ List Pets                                │ ✓ Pass               │ All pets retrieved        │
│ Get Specific Pet                         │ ✓ Pass               │ Pet details retrieved     │
│ Update Pet (PATCH)                       │ ✓ Pass               │ Pet info updated          │
│ Multiple Pets                            │ ✓ Pass               │ Created 3 pets            │
│ Authorization                            │ ✓ Pass               │ Access control working    │
│ Input Validation                         │ ✓ Pass               │ Invalid data rejected     │
│ Cleanup                                  │ ✓ Pass               │ Deleted 4 pets            │
└──────────────────────────────────────────┴──────────────────────┴───────────────────────────┘

All 11 user service tests passed!

## Final Cleanup

In [15]:
# Clear session
session.cookies.clear()
console.print("[blue]Session cleared[/blue]")


Session cleared